In [1]:
# Data prep
import os
import pandas as pd
import numpy as  np
import sqlite3
import warnings
warnings.filterwarnings('ignore')

### Users

In [2]:
users = pd.read_csv('prepared/users.csv')
users

,Unnamed: 0,id,name,address,postalcode,city,birthday,phone,country,active,excluded,age
0,0,1,Breena Connow,16 Acker Alley,NaN,Villa Serrano,1978-12-31,722-967-3477,Bolivia,True,False,44
1,1,2,Harris Dunkerton,8356 Talisman Crossing,99871,Inari,1990-12-03,909-642-7516,Finland,True,False,32
2,2,3,Merrie Grindle,3 Homewood Junction,NaN,Cengjia,1944-10-13,876-111-3766,China,False,True,78
3,3,4,Arlan Morstatt,05 Straubel Place,13315-000,Cabreúva,1945-03-03,870-897-0217,Brazil,True,False,77
4,4,5,Kiele Dedrick,947 Mallard Drive,3515-517,Bodiosa a Velha,1950-02-26,879-743-0896,Portugal,True,False,72
...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,996,Jere Matys,34 Park Meadow Crossing,195047,Florencia,1960-01-06,967-680-6203,Colombia,True,False,62
996,996,997,Bax Levick,17468 Kropf Court,NaN,Gaotang,1961-12-21,399-742-6576,China,True,False,61
997,997,998,Hermine Bowerbank,4100 Brown Avenue,32-060,Rzozów,1989-12-06,240-137-7616,Poland,False,True,33
998,998,999,Beaufort Caulier,72724 Rigney Hill,11320,Kandana,1999-02-13,961-971-2453,Sri Lanka,False,True,23


In [3]:
f = open("populate.sql", "w", encoding="utf-8")

for index, row in users.iterrows():
    f.write("INSERT INTO \"User\" VALUES ('" 
            + str(row['id']) + "','" 
            + str(row['name']) + "','"
            + str(row['address']) + "','"
            + str(row['postalcode']) + "','"
            + str(row['city']) + "','"
            + str(row['birthday']) + "','"
            + str(row['age']) + "','"
            + str(row['phone']) + "','"
            + str(row['country']) + "','"
            + str(row['active']) + "','"
            + str(row['excluded']) + "');\n")
f.write('\n')
f.close()

### Events

In [4]:
baseball = pd.read_csv('prepared/americanfootball.csv')
basketball = pd.read_csv('prepared/basketball.csv')
football = pd.read_csv('prepared/football.csv')
hockey = pd.read_csv('prepared/hockey.csv')
events = football.append(baseball).append(basketball).append(hockey)
events = events.drop(columns=['Unnamed: 0', 'ID'])
events = events.sort_values(by='Date')
events = events.rename(columns={"Score": "Result"})
events['Result'] = events['Result'].apply(lambda x: str(x).replace("–", "-"))
events.loc[events["Active"] == True, "Result"] = np.NaN
events.loc[events["Result"] == 'nan', "Result"] = np.NaN
events.insert(0, 'ID', range(0, 0 + len(events)))
events

,ID,Category,Date,Result,Name,Active
0,0,Football,2022-08-05,4-0,Benfica VS Arouca,False
1047,1,Football,2022-08-05,2-1,Lyon VS Ajaccio,False
477,2,Football,2022-08-05,0-2,Crystal Palace VS Arsenal,False
234,3,Football,2022-08-05,1-6,Eint Frankfurt VS Bayern Munich,False
1049,4,Football,2022-08-06,0-5,Clermont Foot VS Paris S G,False
...,...,...,...,...,...,...
770,4663,Football,2023-04-08,NaN,Leeds United VS Crystal Palace,True
768,4664,Football,2023-04-08,NaN,Aston Villa VS Nott ham Forest,True
767,4665,Football,2023-04-08,NaN,Fulham VS West Ham,True
475,4666,Football,2023-04-08,NaN,Bochum VS Stuttgart,True


In [5]:
f = open("populate.sql", "a", encoding="utf-8")

for index, row in events.iterrows():
    f.write("INSERT INTO \"Event\" VALUES ('" 
            + str(row['ID']) + "','" 
            + str(row['Name']) + "','"
            + str(row['Date']) + "','"
            + str(row['Category']) + "','"
            + str(row['Active']) + "','"
            + str(row['Result']) + "');\n")
f.write('\n')
f.close()

### Market

In [6]:
import random

eventIDs = []
for i in range(3000):
    eventIDs.append(random.randint(0,4075))

market = pd.DataFrame(eventIDs, columns =['EventID'])
market.insert(0, 'ID', range(0, 0 + len(market)))
market['Type'] = np.random.choice(['Score', 'Winner'], len(market))
market

,ID,EventID,Type
0,0,101,Winner
1,1,1679,Score
2,2,1478,Winner
3,3,3224,Winner
4,4,4008,Winner
...,...,...,...
2995,2995,535,Winner
2996,2996,3173,Winner
2997,2997,1512,Winner
2998,2998,1111,Score


In [7]:
f = open("populate.sql", "a", encoding="utf-8")

for index, row in market.iterrows():
    f.write("INSERT INTO \"Market\" VALUES ('" 
            + str(row['ID']) + "','" 
            + str(row['Type']) + "','"
            + str(row['EventID']) + "');\n")
f.write('\n')
f.close()

### Contract

In [8]:
marketIDs = []
for i in range(3000):
    marketIDs.append(random.randint(0,2999))

contract = pd.DataFrame(marketIDs, columns =['MarketID'])
contract.insert(0, 'ID', range(0, 0 + len(contract)))
contract['Type'] = np.random.choice(['Home', 'Draw', 'Away'], len(contract))
contract

,ID,MarketID,Type
0,0,799,Away
1,1,2645,Away
2,2,1784,Draw
3,3,1920,Away
4,4,2269,Draw
...,...,...,...
2995,2995,599,Home
2996,2996,828,Away
2997,2997,962,Home
2998,2998,367,Home


In [9]:
f = open("populate.sql", "a", encoding="utf-8")

for index, row in contract.iterrows():
    f.write("INSERT INTO \"Contract\" VALUES ('" 
            + str(row['ID']) + "','" 
            + str(row['Type']) + "','"
            + str(row['MarketID']) + "');\n")
f.write('\n')
f.close()

### Order

In [10]:
userIDs = []
contractsIDs = []

nonexcludedUsers = users[users['excluded'] == False]
activeUsers = nonexcludedUsers[nonexcludedUsers['active'] == True]

activeUsersIDs = activeUsers["id"].values.tolist()

for i in range(3000):
    userIDs.append(random.randint(1,3000))
    contractsIDs.append(random.randint(0,2999))
    
order = pd.DataFrame(userIDs, columns =['userID'])
order['contractID'] = contractsIDs
order.insert(0, 'ID', range(0, 0 + len(order)))

order

,ID,userID,contractID
0,0,1342,2410
1,1,687,420
2,2,2142,2322
3,3,663,1322
4,4,1658,2723
...,...,...,...
2995,2995,1810,535
2996,2996,1704,2759
2997,2997,2293,283
2998,2998,1557,549


In [11]:
contractsInOrders = contract[contract['ID'].isin(contractsIDs)]
marketIDS = contractsInOrders["MarketID"].values.tolist()
contractsInOrders

,ID,MarketID,Type
1,1,2645,Away
2,2,1784,Draw
3,3,1920,Away
4,4,2269,Draw
5,5,967,Away
...,...,...,...
2992,2992,1696,Draw
2994,2994,1693,Draw
2996,2996,828,Away
2997,2997,962,Home


In [12]:
eventsInMarkets = market[market['ID'].isin(marketIDs)]
eventsInMarkets = pd.merge(eventsInMarkets, events, left_on="EventID", right_on="ID")
eventsInMarkets = eventsInMarkets.drop(columns = ['ID_y'])
eventsInMarkets.rename(columns = {'ID_x':'MarketID'}, inplace = True)
eventsInMarkets

,MarketID,EventID,Type,Category,Date,Result,Name,Active
0,0,101,Winner,Football,2022-08-20,2-1,Marseille VS Nantes,False
1,1070,101,Winner,Football,2022-08-20,2-1,Marseille VS Nantes,False
2,1,1679,Score,Hockey,2022-11-17,6 - 4,Columbus Blue Jackets - Montreal Canadiens,False
3,3,3224,Winner,Basketball,2023-02-01,NaN,San Antonio Spurs - Sacramento Kings,True
4,2627,3224,Score,Basketball,2023-02-01,NaN,San Antonio Spurs - Sacramento Kings,True
...,...,...,...,...,...,...,...,...
1898,2986,1441,Score,Hockey,2022-11-07,5 - 4,Washington Capitals - Edmonton Oilers,False
1899,2989,2090,Score,Basketball,2022-12-09,NaN,Charlotte Hornets - New York Knicks,True
1900,2995,535,Winner,Football,2022-10-01,2-0,West Ham VS Wolves,False
1901,2996,3173,Winner,Football,2023-01-29,NaN,Nice VS Lille,True


In [27]:
status = []
for index, row in order.iterrows():
    marketid = contract.loc[contract['ID'] == row['contractID'], 'MarketID'].iloc[0]
    #active = eventsInMarkets.loc[eventsInMarkets['MarketID'] == marketid, 'Active'].iloc[0]
    result = str(eventsInMarkets.loc[eventsInMarkets['MarketID'] == marketid, 'Result'].iloc[0])
    #print(row['contractID'])
   
    if result == np.nan or result == 'nan':
        #print('here')
        status.append('Pending')
    else:
        #print('or here')
        values = str(result).split('-')
        bet = contractsInOrders.loc[contractsInOrders['ID'] == row['contractID'], 'Type'].iloc[0]
        if bet == 'Draw':
            if int(values[0]) == int(values[1]):
                status.append('Won')
            else: 
                status.append('Lost')
        elif bet == 'Home':
            if int(values[0]) > int(values[1]):
                status.append('Won')
            else: 
                status.append('Lost')
        else:
            if int(values[0]) < int(values[1]):
                status.append('Won')
            else: 
                status.append('Lost')
print(status)

['Pending', 'Lost', 'Pending', 'Pending', 'Pending', 'Lost', 'Pending', 'Lost', 'Pending', 'Pending', 'Lost', 'Lost', 'Pending', 'Pending', 'Pending', 'Won', 'Pending', 'Lost', 'Pending', 'Pending', 'Pending', 'Lost', 'Lost', 'Pending', 'Pending', 'Pending', 'Lost', 'Lost', 'Won', 'Pending', 'Lost', 'Pending', 'Lost', 'Pending', 'Won', 'Pending', 'Pending', 'Lost', 'Lost', 'Pending', 'Lost', 'Won', 'Pending', 'Lost', 'Lost', 'Won', 'Pending', 'Pending', 'Pending', 'Pending', 'Won', 'Won', 'Lost', 'Pending', 'Won', 'Lost', 'Lost', 'Pending', 'Lost', 'Lost', 'Pending', 'Pending', 'Pending', 'Lost', 'Won', 'Lost', 'Pending', 'Won', 'Lost', 'Lost', 'Pending', 'Pending', 'Pending', 'Pending', 'Won', 'Pending', 'Pending', 'Pending', 'Pending', 'Pending', 'Pending', 'Pending', 'Lost', 'Pending', 'Pending', 'Lost', 'Lost', 'Lost', 'Pending', 'Pending', 'Pending', 'Lost', 'Pending', 'Lost', 'Won', 'Pending', 'Pending', 'Won', 'Pending', 'Won', 'Pending', 'Lost', 'Pending', 'Lost', 'Lost', 'Pend

In [29]:
order['Status'] = status
order['Status'].value_counts()

Pending    1515
Lost        960
Won         525
Name: Status, dtype: int64

In [31]:
initialValues = []
currentValues = []

for i in range(3000):
    initialValues.append(random.randint(1,200))

for i in range(3000):
    currentValues.append(random.randint(0,initialValues[i]))

In [33]:
order['initialValue'] = initialValues
order['currentValue'] = currentValues
order

,ID,userID,contractID,Status,initialValues,currentValues,initialValue,currentValue
0,0,1342,2410,Pending,21,6,21,6
1,1,687,420,Lost,8,4,8,4
2,2,2142,2322,Pending,168,68,168,68
3,3,663,1322,Pending,70,8,70,8
4,4,1658,2723,Pending,9,4,9,4
...,...,...,...,...,...,...,...,...
2995,2995,1810,535,Lost,49,11,49,11
2996,2996,1704,2759,Pending,80,55,80,55
2997,2997,2293,283,Pending,125,40,125,40
2998,2998,1557,549,Lost,87,75,87,75


In [35]:
f = open("populate.sql", "a", encoding="utf-8")

for index, row in order.iterrows():
    f.write("INSERT INTO \"Order\" VALUES ('" 
            + str(row['ID']) + "','" 
            + str(row['userID']) + "','"
            + str(row['contractID']) + "','" 
            + str(row['initialValue']) + "','"
            + str(row['currentValue']) + "','" 
            + str(row['Status']) + "');\n")
f.write('\n')
f.close()

### Deal

In [49]:
t_order = order
t_events = events
t_contracts = contract

temp = eventsInMarkets[['MarketID', 'EventID']]

merged1 = pd.merge(temp, t_contracts, left_on="MarketID", right_on="MarketID")
merged1 = merged1.rename(columns={'ID': 'ContractID'})

data = pd.merge(merged1, t_order, left_on="ContractID", right_on="contractID")
data = data.rename(columns={'ID': 'OrderID'})
data = data.drop(['userID', 'initialValues', 'currentValues'], axis=1)
data

,MarketID,EventID,ContractID,Type,OrderID,contractID,Status,initialValue,currentValue
0,0,101,2536,Home,675,2536,Won,116,31
1,1070,101,718,Draw,1109,718,Lost,10,5
2,1,1679,787,Draw,1721,787,Lost,105,1
3,3,3224,2116,Away,713,2116,Pending,43,25
4,2627,3224,1366,Away,2548,1366,Pending,167,39
...,...,...,...,...,...,...,...,...,...
2995,2995,535,2243,Away,1607,2243,Lost,26,4
2996,2996,3173,2354,Home,1492,2354,Pending,43,15
2997,2996,3173,2354,Home,2010,2354,Pending,116,47
2998,2998,1111,1010,Home,1350,1010,Lost,149,32
